<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: Юрий Исаков и Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Тема 4. Линейные модели классификации и регрессии
## <center>  Практика. Идентификация пользователя с помощью логистической регрессии

Тут мы воспроизведем парочку бенчмарков нашего соревнования и вдохновимся побить третий бенчмарк, а также остальных участников. Веб-формы для отправки ответов тут не будет, ориентир – [leaderboard](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard) соревнования.

In [1]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

### 1. Загрузка и преобразование данных
Зарегистрируйтесь на [Kaggle](www.kaggle.com), если вы не сделали этого раньше, зайдите на [страницу](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования и скачайте данные. Первым делом загрузим обучающую и тестовую выборки и посмотрим на данные.

In [2]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv('../../data/alice/train_sessions.csv',
                       index_col='session_id')
test_df = pd.read_csv('../../data/alice/test_sessions.csv',
                      index_col='session_id')

In [3]:
# приведем колонки time1, ..., time10 к временному формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.reset_index().sort_values(by='time1').reset_index(drop=True)
test_df = test_df.reset_index().sort_values(by='time1').reset_index(drop=True)

# посмотрим на заголовок обучающей выборки
display(train_df.head(5))
display(train_df.tail(5))

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
0,21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
1,54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
2,77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
3,114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
4,146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
253556,12224,50,2014-04-30 23:33:48,50.0,2014-04-30 23:33:49,48.0,2014-04-30 23:33:52,49.0,2014-04-30 23:33:52,48.0,...,2014-04-30 23:33:53,52.0,2014-04-30 23:33:54,49.0,2014-04-30 23:33:54,303.0,2014-04-30 23:33:57,304.0,2014-04-30 23:34:00,0
253557,164438,4207,2014-04-30 23:34:15,753.0,2014-04-30 23:34:16,753.0,2014-04-30 23:34:17,52.0,2014-04-30 23:34:18,50.0,...,2014-04-30 23:35:16,3346.0,2014-04-30 23:35:29,3359.0,2014-04-30 23:36:12,3346.0,2014-04-30 23:36:42,38.0,2014-04-30 23:37:13,0
253558,12221,52,2014-04-30 23:38:08,3346.0,2014-04-30 23:38:10,784.0,2014-04-30 23:38:13,784.0,2014-04-30 23:38:18,3346.0,...,2014-04-30 23:38:24,3324.0,2014-04-30 23:38:35,7330.0,2014-04-30 23:38:35,3594.0,2014-04-30 23:38:35,3329.0,2014-04-30 23:38:36,0
253559,156968,3328,2014-04-30 23:38:36,3324.0,2014-04-30 23:38:36,3599.0,2014-04-30 23:38:38,3413.0,2014-04-30 23:38:38,753.0,...,2014-04-30 23:38:40,3599.0,2014-04-30 23:38:40,3359.0,2014-04-30 23:39:07,3359.0,2014-04-30 23:39:08,3346.0,2014-04-30 23:39:53,0
253560,204762,222,2014-04-30 23:39:53,3346.0,2014-04-30 23:39:59,3346.0,2014-04-30 23:40:00,3359.0,2014-04-30 23:40:05,55.0,...,2014-04-30 23:40:05,3346.0,2014-04-30 23:40:05,NaN,NaT,NaN,NaT,NaN,NaT,0


In [4]:
display(test_df.head(5))
display(test_df.tail(5))

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
0,65540,21,2014-05-01 17:14:03,NaN,NaT,NaN,NaT,NaN,NaT,NaN,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
1,64199,23,2014-05-02 07:52:08,66.0,2014-05-02 07:54:08,63.0,2014-05-02 07:54:08,2626.0,2014-05-02 07:55:09,NaN,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
2,2268,979,2014-05-02 07:57:51,73.0,2014-05-02 07:59:34,NaN,NaT,NaN,NaT,NaN,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
3,29734,66,2014-05-02 08:05:16,69.0,2014-05-02 08:05:17,67.0,2014-05-02 08:05:17,70.0,2014-05-02 08:05:17,71.0,...,68.0,2014-05-02 08:05:17,71.0,2014-05-02 08:05:18,70.0,2014-05-02 08:05:18,69.0,2014-05-02 08:05:18,67.0,2014-05-02 08:05:18
4,77048,167,2014-05-02 08:05:32,167.0,2014-05-02 08:05:33,359.0,2014-05-02 08:05:34,167.0,2014-05-02 08:05:34,167.0,...,305.0,2014-05-02 08:09:19,306.0,2014-05-02 08:09:20,306.0,2014-05-02 08:09:22,979.0,2014-05-02 08:09:54,68.0,2014-05-02 08:12:46


,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
82792,73506,2891,2014-12-05 20:07:54,55.0,2014-12-05 20:07:54,NaN,NaT,NaN,NaT,NaN,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
82793,59132,56,2014-12-05 20:55:17,55.0,2014-12-05 20:55:17,56.0,2014-12-05 21:23:38,55.0,2014-12-05 21:23:38,NaN,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
82794,78588,56,2014-12-05 21:54:46,55.0,2014-12-05 21:54:46,NaN,NaT,NaN,NaT,NaN,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
82795,26999,56,2014-12-05 22:26:40,55.0,2014-12-05 22:26:40,56.0,2014-12-05 22:55:10,55.0,2014-12-05 22:55:10,NaN,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
82796,60141,56,2014-12-05 23:26:53,55.0,2014-12-05 23:26:53,56.0,2014-12-05 23:56:23,55.0,2014-12-05 23:56:23,NaN,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT


In [5]:
train_df.time1.apply(lambda x: '{}-{}'.format(x.year, x.month)).value_counts()

2014-3     56250
2014-2     55563
2014-1     35869
2013-11    34493
2014-4     31204
2013-12    16552
2013-4      3997
2013-2      3929
2013-9      3464
2013-3      3337
2013-5      3229
2013-10     2788
2013-6      2094
2013-1       363
2013-8       290
2013-7       139
Name: time1, dtype: int64

In [6]:
test_df.time1.apply(lambda x: '{}-{}'.format(x.year, x.month)).value_counts()

2014-5     25680
2014-12    12388
2014-7     10837
2014-11     9897
2014-10     8157
2014-8      7846
2014-6      4365
2014-9      3627
Name: time1, dtype: int64

In [7]:
foo = train_df.copy()
foo['h'] = foo.time1.apply(lambda x: '{}-{}'.format(x.year, x.month))
foo[['h', 'target']].groupby('h').agg([len, np.mean]).reset_index().sort_values(('target', 'mean'))

h target          
               len      mean
0    2013-1    363  0.000000
1   2013-10   2788  0.000000
5    2013-3   3337  0.000000
7    2013-5   3229  0.000000
8    2013-6   2094  0.000000
9    2013-7    139  0.000000
10   2013-8    290  0.000000
12   2014-1  35869  0.003596
14   2014-3  56250  0.007111
13   2014-2  55563  0.007379
3   2013-12  16552  0.008096
6    2013-4   3997  0.009507
15   2014-4  31204  0.009678
2   2013-11  34493  0.012930
4    2013-2   3929  0.015526
11   2013-9   3464  0.108834

In [8]:
foo = train_df.copy()
foo['h'] = foo.time1.apply(lambda x: '{}'.format(x.hour))
foo[['h', 'target']].groupby('h').agg(np.mean).reset_index().sort_values('target')

,h,target
0,10,0.000000
14,7,0.000000
13,23,0.000000
12,22,0.000000
11,21,0.000000
10,20,0.000000
15,8,0.000000
9,19,0.000000
1,11,0.000097
4,14,0.000146


In [13]:
foo = train_df.copy()
foo['h'] = foo.time1.apply(lambda x: '{}'.format(x.year))
foo[['h', 'target']].groupby('h').agg(np.mean).reset_index().sort_values('target')

,h,target
1,2014,0.006937
0,2013,0.014141


In [11]:
train_df.time1.apply(lambda x: '{}'.format(x.hour)).value_counts()

10    33676
9     31741
11    30798
14    27306
8     25369
13    22552
15    21640
16    18739
12    17420
17    12830
18     3898
21     1705
19     1540
22     1467
23     1339
20     1200
7       341
Name: time1, dtype: int64

In [12]:
test_df.time1.apply(lambda x: '{}'.format(x.hour)).value_counts()

10    11114
11    10677
14     9597
9      9170
13     7840
15     7436
8      6297
16     6263
12     5187
17     4808
18     1891
19      767
20      593
21      424
22      290
7       259
23      184
Name: time1, dtype: int64

В обучающей выборке содержатся следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - target – целевая переменная, 1 для сессий Элис, 0 для сессий других пользователей
    
Сессии пользователей выделены таким образом, что они не могут быть длиннее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд либо когда сессия заняла по времени более 30 минут.

В таблице встречаются пропущенные значения, это значит, что сессия состоит менее, чем из 10 сайтов. Заменим пропущенные значения нулями и приведем признаки к целому типу. Также загрузим словарь сайтов и посмотрим, как он выглядит:

In [15]:
# приведем колонки site1, ..., site10 к целочисленному формату и заменим пропуски нулями
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# загрузим словарик сайтов
with open(r"../../data/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(list(site_dict.keys()), 
                          index=list(site_dict.values()), 
                          columns=['site'])
print(u'всего сайтов:', sites_dict_df.shape[0])
sites_dict_df.head()

всего сайтов: 48371


,site
16670,navi-gaming.com
46794,popin.espritservices.gdfsuez-dolcevita.fr
8251,792817.r.msn.com
22976,ws3.lefigaro.fr
532,i.stack.imgur.com


Выделим целевую переменную и объединим выборки, чтобы вместе привести их к разреженному формату.

In [16]:
site_dict_inv = dict()
for k,v in site_dict.items():
    site_dict_inv[v] = k

In [17]:
# наша целевая переменная
y_train = train_df['target'].values

# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]

In [18]:
full_df

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
0,21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,NaT,0,NaT,0,...,0,NaT,0,NaT,0,NaT,0,NaT,0,NaT
1,54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,...,0,NaT,0,NaT,0,NaT,0,NaT,0,NaT
2,77292,946,2013-01-12 08:50:13,946,2013-01-12 08:50:14,951,2013-01-12 08:50:15,946,2013-01-12 08:50:15,946,...,945,2013-01-12 08:50:16,948,2013-01-12 08:50:16,784,2013-01-12 08:50:16,949,2013-01-12 08:50:17,946,2013-01-12 08:50:17
3,114021,945,2013-01-12 08:50:17,948,2013-01-12 08:50:17,949,2013-01-12 08:50:18,948,2013-01-12 08:50:18,945,...,946,2013-01-12 08:50:18,947,2013-01-12 08:50:19,945,2013-01-12 08:50:19,946,2013-01-12 08:50:19,946,2013-01-12 08:50:20
4,146670,947,2013-01-12 08:50:20,950,2013-01-12 08:50:20,948,2013-01-12 08:50:20,947,2013-01-12 08:50:21,950,...,952,2013-01-12 08:50:21,946,2013-01-12 08:50:21,951,2013-01-12 08:50:22,946,2013-01-12 08:50:22,947,2013-01-12 08:50:22
5,242171,952,2013-01-12 08:50:22,947,2013-01-12 08:50:23,953,2013-01-12 08:50:23,946,2013-01-12 08:50:23,947,...,946,2013-01-12 08:50:24,953,2013-01-12 08:50:24,955,2013-01-12 08:50:24,946,2013-01-12 08:50:25,947,2013-01-12 08:50:25
6,57157,953,2013-01-12 08:50:25,947,2013-01-12 08:50:26,946,2013-01-12 08:50:26,953,2013-01-12 08:50:26,955,...,947,2013-01-12 08:50:27,953,2013-01-12 08:50:27,946,2013-01-12 08:50:27,953,2013-01-12 08:50:28,1033,2013-01-12 08:50:28
7,240201,946,2013-01-12 08:50:28,947,2013-01-12 08:50:28,954,2013-01-12 08:50:28,953,2013-01-12 08:50:29,946,...,954,2013-01-12 08:50:29,946,2013-01-12 08:50:30,956,2013-01-12 08:50:30,957,2013-01-12 08:50:31,956,2013-01-12 08:50:31
8,210686,946,2013-01-12 08:50:31,956,2013-01-12 08:50:32,946,2013-01-12 08:50:32,946,2013-01-12 08:50:33,955,...,954,2013-01-12 08:50:33,946,2013-01-12 08:50:34,946,2013-01-12 08:50:35,946,2013-01-12 08:50:36,948,2013-01-12 08:50:36
9,98804,948,2013-01-12 08:50:37,946,2013-01-12 08:50:37,948,2013-01-12 08:50:38,784,2013-01-12 08:50:49,49,...,53,2013-01-12 08:51:03,812,2013-01-12 08:51:03,982,2013-01-12 08:51:03,52,2013-01-12 08:51:03,52,2013-01-12 08:51:04


Для самой первой модели будем использовать только посещенные сайты в сессии (но не будем обращать внимание на временные признаки). За таким выбором данных для модели стоит такая идея:  *у Элис есть свои излюбленные сайты, и чем чаще вы видим эти сайты в сессии, тем выше вероятность, что это сессия Элис и наоборот.*

Подготовим данные, из всей таблицы выберем только признаки `site1, site2, ... , site10`. Напомним, что пропущенные значения заменены нулем. Вот как выглядят первые строки таблицы:

In [19]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
0,56,55,0,0,0,0,0,0,0,0
1,56,55,56,55,0,0,0,0,0,0
2,946,946,951,946,946,945,948,784,949,946
3,945,948,949,948,945,946,947,945,946,946
4,947,950,948,947,950,952,946,951,946,947


Сессии представляют собой последовательность индексов сайтов и данные в таком виде неудобны для линейных методов. В соответствии с нашей гипотезой (у Элис есть излюбленные сайты) надо преобразовать эту таблицу таким образом, чтобы каждому возможному сайту соответствовал свой отдельный признак (колонка), а его значение равнялось бы количеству посещений этого сайта в сессии. Это делается в две строчки:

In [20]:
from scipy.sparse import csr_matrix

In [21]:
csr_matrix?

In [22]:
# последовательность с индексами
sites_flatten = full_sites.values.flatten()

In [23]:
from collections import Counter
from collections import defaultdict

In [25]:
sites_counter = Counter()
tokens_counter = Counter()

for i in tqdm_notebook(range(full_sites.shape[0]), total=full_sites.shape[0]):
    d = defaultdict(int)
    for j in range(i * full_sites.shape[1], (i + 1) * full_sites.shape[1]):
        if sites_flatten[j] != 0:
            d[sites_flatten[j]] += 1
    sites_counter.update(d.keys())
    for e in d.keys():
        tokens_counter.update(set(site_dict_inv[e].split('.')))

A Jupyter Widget

In [26]:
len(sites_counter), len(tokens_counter)

(48371, 40677)

In [27]:
idfs_sites = dict()
for e in sites_counter.items():
    idfs_sites[e[0]] = np.log(full_sites.shape[0] / e[1])

In [28]:
[(site_dict_inv[e[0]], e[1]) for e in sorted(sites_counter.items(), key=lambda x: -x[1])[:5]]

[('www.google.fr', 84205),
 ('www.google.com', 76631),
 ('apis.google.com', 56597),
 ('www.facebook.com', 52470),
 ('accounts.google.com', 27645)]

In [33]:
domains = Counter()
domains.update([site_dict_inv[e].split('.')[-1] for e in site_dict_inv.keys()])
domains.most_common()

[('com', 22284),
 ('net', 9512),
 ('fr', 8558),
 ('org', 2208),
 ('edu', 525),
 ('cn', 466),
 ('ca', 441),
 ('us', 364),
 ('ru', 311),
 ('uk', 309),
 ('ch', 274),
 ('be', 265),
 ('de', 250),
 ('info', 237),
 ('eu', 227),
 ('au', 154),
 ('tv', 140),
 ('es', 105),
 ('gov', 94),
 ('it', 81),
 ('io', 75),
 ('nl', 63),
 ('tt', 60),
 ('me', 58),
 ('jp', 55),
 ('ro', 53),
 ('ms', 52),
 ('br', 50),
 ('biz', 42),
 ('in', 40),
 ('co', 39),
 ('ma', 39),
 ('nz', 36),
 ('se', 34),
 ('dk', 30),
 ('fm', 30),
 ('cc', 29),
 ('pl', 27),
 ('cz', 23),
 ('at', 22),
 ('no', 22),
 ('cat', 22),
 ('ie', 20),
 ('ua', 19),
 ('ws', 18),
 ('fi', 17),
 ('hu', 17),
 ('dz', 16),
 ('kr', 14),
 ('pe', 14),
 ('int', 13),
 ('lu', 13),
 ('is', 13),
 ('za', 13),
 ('gr', 12),
 ('travel', 12),
 ('hk', 11),
 ('pt', 10),
 ('ar', 9),
 ('im', 9),
 ('to', 9),
 ('pro', 9),
 ('mx', 8),
 ('re', 8),
 ('name', 8),
 ('cx', 8),
 ('su', 7),
 ('tn', 7),
 ('cl', 7),
 ('sn', 7),
 ('kz', 7),
 ('tw', 6),
 ('by', 6),
 ('sk', 6),
 ('nc', 6),
 (

In [64]:
foo = train_df.copy()
foo['domains'] = foo[sites].apply(lambda x: ' '.join([site_dict_inv.get(e, '').split('.')[-1] for e in x]), axis=1)
#foo['h'] = foo.time1.apply(lambda x: '{}-{}'.format(x.year, x.month))
#foo[['h', 'target']].groupby('h').agg([len, np.mean]).reset_index().sort_values(('target', 'mean'))
foo

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,site7,time7,site8,time8,site9,time9,site10,time10,target,domains
0,21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,NaT,0,NaT,0,...,0,NaT,0,NaT,0,NaT,0,NaT,0,com com
1,54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,...,0,NaT,0,NaT,0,NaT,0,NaT,0,com com com com
2,77292,946,2013-01-12 08:50:13,946,2013-01-12 08:50:14,951,2013-01-12 08:50:15,946,2013-01-12 08:50:15,946,...,948,2013-01-12 08:50:16,784,2013-01-12 08:50:16,949,2013-01-12 08:50:17,946,2013-01-12 08:50:17,0,org org org org org com com com com org
3,114021,945,2013-01-12 08:50:17,948,2013-01-12 08:50:17,949,2013-01-12 08:50:18,948,2013-01-12 08:50:18,945,...,947,2013-01-12 08:50:19,945,2013-01-12 08:50:19,946,2013-01-12 08:50:19,946,2013-01-12 08:50:20,0,com com com com com org com com org org
4,146670,947,2013-01-12 08:50:20,950,2013-01-12 08:50:20,948,2013-01-12 08:50:20,947,2013-01-12 08:50:21,950,...,946,2013-01-12 08:50:21,951,2013-01-12 08:50:22,946,2013-01-12 08:50:22,947,2013-01-12 08:50:22,0,com org com com org net org org org com
5,242171,952,2013-01-12 08:50:22,947,2013-01-12 08:50:23,953,2013-01-12 08:50:23,946,2013-01-12 08:50:23,947,...,953,2013-01-12 08:50:24,955,2013-01-12 08:50:24,946,2013-01-12 08:50:25,947,2013-01-12 08:50:25,0,net com org org com org org net org com
6,57157,953,2013-01-12 08:50:25,947,2013-01-12 08:50:26,946,2013-01-12 08:50:26,953,2013-01-12 08:50:26,955,...,953,2013-01-12 08:50:27,946,2013-01-12 08:50:27,953,2013-01-12 08:50:28,1033,2013-01-12 08:50:28,0,org com org org net com org org org net
7,240201,946,2013-01-12 08:50:28,947,2013-01-12 08:50:28,954,2013-01-12 08:50:28,953,2013-01-12 08:50:29,946,...,946,2013-01-12 08:50:30,956,2013-01-12 08:50:30,957,2013-01-12 08:50:31,956,2013-01-12 08:50:31,0,org com net org org net org net net net
8,210686,946,2013-01-12 08:50:31,956,2013-01-12 08:50:32,946,2013-01-12 08:50:32,946,2013-01-12 08:50:33,955,...,946,2013-01-12 08:50:34,946,2013-01-12 08:50:35,946,2013-01-12 08:50:36,948,2013-01-12 08:50:36,0,org net org org net net org org org com
9,98804,948,2013-01-12 08:50:37,946,2013-01-12 08:50:37,948,2013-01-12 08:50:38,784,2013-01-12 08:50:49,49,...,812,2013-01-12 08:51:03,982,2013-01-12 08:51:03,52,2013-01-12 08:51:03,52,2013-01-12 08:51:04,0,com org com com com com com com com com


In [71]:
for e,_ in domains.most_common(20):
    foo['isDomain_' + e] = foo['domains'].apply(lambda x: int(x.find(e) != -1))
    display(foo[['isDomain_' + e, 'target']].groupby('isDomain_' + e).agg(np.mean).reset_index().sort_values('target'))

,isDomain_com,target
0,0,0.005095
1,1,0.009599


,isDomain_net,target
0,0,0.008707
1,1,0.010137


,isDomain_fr,target
1,1,0.007906
0,0,0.010129


,isDomain_org,target
1,1,0.003747
0,0,0.010031


,isDomain_edu,target
1,1,0.000000
0,0,0.009105


...

,isDomain_au,target
1,1,0.000000
0,0,0.009074


,isDomain_tv,target
0,0,0.008967
1,1,0.033368


,isDomain_es,target
0,0,0.009049
1,1,0.016216


,isDomain_gov,target
1,1,0.00000
0,0,0.00953


,isDomain_it,target
0,0,0.009039
1,1,0.030973


In [72]:
foo['h'] = foo['domains'].apply(lambda x: len(set(x.strip().split(' '))))
display(foo[['h', 'target']].groupby('h').agg(np.mean).reset_index().sort_values('target'))

,h,target
7,8,0.000000
1,2,0.006293
2,3,0.008439
0,1,0.012056
3,4,0.015558
5,6,0.027027
4,5,0.037657
6,7,0.038462


In [29]:
sorted(tokens_counter.items(), key=lambda x: -x[1])[:5]

[('com', 1260961),
 ('google', 495022),
 ('www', 436546),
 ('fr', 316167),
 ('net', 149289)]

In [34]:
n_sites = len(sites_counter)

In [131]:
datas = []
rows = []
cols = []
for i in tqdm_notebook(range(full_sites.shape[0]), total=full_sites.shape[0]):
    d = defaultdict(int)
    zc = 0
    q = 0
    for j in range(i * full_sites.shape[1], (i + 1) * full_sites.shape[1]):
        if sites_flatten[j] != 0:
            d[sites_flatten[j]] += 1
        else:
            zc += 1
        
        datas.append(len(d) / (q - zc + 1))
        rows.append(i)
        cols.append(n_sites + q + 1)
        q += 1
    #d[0] = zc
    for k, v in d.items():
        datas.append(idfs_sites[k] / 10)
        rows.append(i)
        cols.append(k)
full_sites_sparse = csr_matrix((datas, (rows, cols)))

A Jupyter Widget

In [132]:
full_sites_sparse.shape

(336358, 48382)

In [133]:
# искомая матрица

full_sites_sparse_0 = csr_matrix(([1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0] + 10, 10)))[:, 1:]

Еще один плюс использования разреженных матриц в том, что для них имеются специальные реализации как матричных операций, так и алгоритмов машинного обучения, что подчас позволяет ощутимо ускорить операции за счет особенностей структуры данных. Это касается и логистической регрессии. Вот теперь у нас все готово для построения нашей первой модели.

### 2. Построение первой модели

Итак, у нас есть алгоритм и данные для него, построим нашу первую модель, воспользовавшись релизацией [логистической регрессии](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) из пакета `sklearn` с параметрами по умолчанию. Первые 90% данных будем использовать для обучения (обучающая выборка отсортирована по времени), а оставшиеся 10% для проверки качества (validation). 

**Напишите простую функцию, которая будет возвращать качество модели на отложенной выборке, и обучите наш первый классификатор**.

In [16]:
LogisticRegression?

In [162]:
def get_auc_lr_valid(X, y, C=1.0, ratio = 0.9, seed=17, penalty='l1', lr=None):
    '''
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state 
              логистической регрессии
    '''
    if lr is None:
        lr = LogisticRegression(C=C, random_state=seed, penalty=penalty, solver='liblinear')#, class_weight='balanced')
    ntr = int(X.shape[0] * ratio)
    lr.fit(X[:ntr, :], y[:ntr])
    pred_te = lr.predict_proba(X[ntr:, :])[:, 1]
    pred_tr = lr.predict_proba(X[:ntr, :])[:, 1]
    return roc_auc_score(y[ntr:], pred_te), roc_auc_score(y[:ntr], pred_tr)
    # Ваш код здесь

**Посмотрите, какой получился ROC AUC на отложенной выборке.**

In [40]:
full_sites_sparse.shape, idx_split, len(y_train)

((336358, 48382), 253561, 253561)

In [41]:
train_df.target.value_counts()

0    251264
1      2297
Name: target, dtype: int64

In [134]:
# Ваш код здесь
for C in [0.05, 0.1, 0.5]:
    print(C, get_auc_lr_valid(full_sites_sparse[:idx_split,:], y_train, C=C, ratio=0.8, seed=17))

0.05 0.822291890083
0.1 0.864334245006
0.5 0.918626817924


In [49]:
for C in [0.1, 0.3, 0.4, 0.5]:
    print(C, get_auc_lr_valid(full_sites_sparse_0[:idx_split,:], y_train, C=C, ratio=0.9, seed=17))

0.1 0.908722355866
0.3 0.916903890256
0.4 0.917168889308
0.5 0.917011142791


Будем считать эту модель нашей первой отправной точкой (baseline). Для построения модели для прогноза на тестовой выборке **необходимо обучить модель заново уже на всей обучающей выборке** (пока наша модель обучалась лишь на части данных), что повысит ее обобщающую способность:

In [117]:
# функция для записи прогнозов в файл
def write_to_submission_file(predicted_labels, index, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = index,
                                columns=[target]).reset_index()
    predicted_df.columns = ['session_id', 'target']
    predicted_df.sort_values('session_id').to_csv(out_file, index=False)

**Обучите модель на всей выборке, сделайте прогноз для тестовой выборки и сделайте посылку в соревновании**.

In [52]:
# Ваш код здесь
lr = LogisticRegression(C=0.5, random_state=17)
X = full_sites_sparse[:idx_split,:]
lr.fit(X, y_train)
pred_labels = lr.predict_proba(full_sites_sparse[idx_split:,:])[:, 1]

In [53]:
roc_auc_score(y_train, lr.predict_proba(full_sites_sparse[:idx_split,:])[:, 1])

0.98455384066622365

In [82]:
#write_to_submission_file(pred_labels, test_df['index'], 'submissions/baseline_1_idf_new.txt')

Если вы выполните эти действия и загрузите ответ на [странице](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования, то воспроизведете первый бенчмарк "Logit".

### 3. Улучшение модели, построение новых признаков

Создайте такой признак, который будет представлять собой число вида ГГГГММ от той даты, когда проходила сессия, например 201407 -- 2014 год и 7 месяц. Таким образом, мы будем учитывать помесячный [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь период предоставленных данных.

In [86]:
StandardScaler?

In [73]:
full_df['domains'] = full_df[sites].apply(lambda x: ' '.join([site_dict_inv.get(e, '').split('.')[-1] for e in x]), axis=1)

In [74]:
full_df.head()

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,time8,site9,time9,site10,time10,start_month,start_time,start_time_sin,start_time_cos,domains
0,21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,NaT,0,NaT,0,...,NaT,0,NaT,0,NaT,-2.079076,-1.512826,0.854912,-0.518773,com com
1,54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,...,NaT,0,NaT,0,NaT,-2.079076,-1.342782,0.774393,-0.632705,com com com com
2,77292,946,2013-01-12 08:50:13,946,2013-01-12 08:50:14,951,2013-01-12 08:50:15,946,2013-01-12 08:50:15,946,...,2013-01-12 08:50:16,949,2013-01-12 08:50:17,946,2013-01-12 08:50:17,-2.079076,-1.273702,0.737277,-0.675590,org org org org org com com com com org
3,114021,945,2013-01-12 08:50:17,948,2013-01-12 08:50:17,949,2013-01-12 08:50:18,948,2013-01-12 08:50:18,945,...,2013-01-12 08:50:19,946,2013-01-12 08:50:19,946,2013-01-12 08:50:20,-2.079076,-1.273702,0.737277,-0.675590,com com com com com org com com org org
4,146670,947,2013-01-12 08:50:20,950,2013-01-12 08:50:20,948,2013-01-12 08:50:20,947,2013-01-12 08:50:21,950,...,2013-01-12 08:50:22,946,2013-01-12 08:50:22,947,2013-01-12 08:50:22,-2.079076,-1.273702,0.737277,-0.675590,com org com com org net org org org com


In [75]:
for e,_ in domains.most_common(20):
    full_df['isDomain_' + e] = full_df['domains'].apply(lambda x: int(x.find(e) != -1))

In [76]:
full_df['domainCnt'] = full_df['domains'].apply(lambda x: len(set(x.strip().split(' '))))

In [77]:
# Ваш код здесь
full_df['start_month'] = full_df['time1'].apply(lambda x: x.year * 100 + x.month)
full_df['start_month'] = StandardScaler().fit_transform(X=full_df['start_month'].astype(float).values.reshape(-1, 1))

In [78]:
full_df['start_time'] = full_df['time1'].apply(lambda x: x.hour * 60 + x.minute)
full_df['start_time_sin'] = full_df['start_time'].apply(lambda x: np.sin(2 * np.pi * x / 1440))
full_df['start_time_cos'] = full_df['start_time'].apply(lambda x: np.cos(2 * np.pi * x / 1440))
full_df['start_time'] = StandardScaler().fit_transform(X=full_df['start_time'].astype(float).values.reshape(-1, 1))

In [79]:
full_df['finish_time'] = full_df[times].apply(max, axis = 1)
full_df['n_sites'] = full_df[sites].apply(lambda x: np.sum(x!=0), axis = 1)

In [80]:
full_df['start_hour'] = full_df['time1'].apply(lambda x: x.hour)

In [81]:
foo = train_df.copy()
foo['h'] = foo.time1.apply(lambda x: '{}'.format(x.hour))
foo[['h', 'target']].groupby('h').agg(np.mean).reset_index().sort_values('target')

,h,target
0,10,0.000000
14,7,0.000000
13,23,0.000000
12,22,0.000000
11,21,0.000000
10,20,0.000000
15,8,0.000000
9,19,0.000000
1,11,0.000097
4,14,0.000146


In [82]:
full_df['is_h1'] = full_df['start_hour'].apply(lambda x: (x >=16  or x <= 18)).astype(int)
full_df['is_h2'] = full_df['start_hour'].apply(lambda x: (x >=12  or x <= 13)).astype(int)
full_df['is_h3'] = full_df['start_hour'].apply(lambda x: (x >=9  or x <= 18)).astype(int)
#full_df['is_morning'] = full_df['start_hour'].apply(lambda x: (x >= 6 and x < 10)).astype(int)
#full_df['is_noon'] = full_df['start_hour'].apply(lambda x: (x >= 10 and x < 17)).astype(int)
#full_df['is_evening'] = full_df['start_hour'].apply(lambda x: (x >= 17 and x < 23)).astype(int)

In [90]:
full_df['duration'] = (full_df['finish_time'] - full_df['time1']).apply(lambda x: x.total_seconds() / 60)
full_df['duration'] /= full_df['n_sites']
full_df['duration'] = StandardScaler().fit_transform(X=full_df['duration'].astype(float).values.reshape(-1, 1))

In [92]:
full_df['n_sites'] /= 10
full_df['start_hour'] /= 24

In [93]:
full_df

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,isDomain_gov,isDomain_it,domainCnt,finish_time,n_sites,start_hour,is_h1,is_h2,is_h3,duration
0,21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,NaT,0,NaT,0,...,0,0,1,2013-01-12 08:05:57,0.2,0.333333,1,1,1,-0.369312
1,54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,...,0,0,1,2013-01-12 09:07:09,0.4,0.333333,1,1,1,9.536076
2,77292,946,2013-01-12 08:50:13,946,2013-01-12 08:50:14,951,2013-01-12 08:50:15,946,2013-01-12 08:50:15,946,...,0,0,2,2013-01-12 08:50:17,1.0,0.333333,1,1,1,-0.360438
3,114021,945,2013-01-12 08:50:17,948,2013-01-12 08:50:17,949,2013-01-12 08:50:18,948,2013-01-12 08:50:18,945,...,0,0,2,2013-01-12 08:50:20,1.0,0.333333,1,1,1,-0.362656
4,146670,947,2013-01-12 08:50:20,950,2013-01-12 08:50:20,948,2013-01-12 08:50:20,947,2013-01-12 08:50:21,950,...,0,0,3,2013-01-12 08:50:22,1.0,0.333333,1,1,1,-0.364875
5,242171,952,2013-01-12 08:50:22,947,2013-01-12 08:50:23,953,2013-01-12 08:50:23,946,2013-01-12 08:50:23,947,...,0,0,3,2013-01-12 08:50:25,1.0,0.333333,1,1,1,-0.362656
6,57157,953,2013-01-12 08:50:25,947,2013-01-12 08:50:26,946,2013-01-12 08:50:26,953,2013-01-12 08:50:26,955,...,0,0,3,2013-01-12 08:50:28,1.0,0.333333,1,1,1,-0.362656
7,240201,946,2013-01-12 08:50:28,947,2013-01-12 08:50:28,954,2013-01-12 08:50:28,953,2013-01-12 08:50:29,946,...,0,0,3,2013-01-12 08:50:31,1.0,0.333333,1,1,1,-0.362656
8,210686,946,2013-01-12 08:50:31,956,2013-01-12 08:50:32,946,2013-01-12 08:50:32,946,2013-01-12 08:50:33,955,...,0,0,3,2013-01-12 08:50:36,1.0,0.333333,1,1,1,-0.358220
9,98804,948,2013-01-12 08:50:37,946,2013-01-12 08:50:37,948,2013-01-12 08:50:38,784,2013-01-12 08:50:49,49,...,0,0,2,2013-01-12 08:51:04,1.0,0.333333,1,1,1,-0.309414


Добавьте новый признак, предварительно отмасштабировав его с помощью `StandardScaler`, и снова посчитайте ROC AUC на отложенной выборке.

In [85]:
full_df.shape, full_sites_sparse.shape

((336358, 54), (336358, 48382))

In [94]:
f_columns = ['start_month', 'start_time', 'start_time_sin', 'start_time_cos']
f_columns += ['isDomain_com', 'isDomain_net',
       'isDomain_fr', 'isDomain_org', 'isDomain_edu', 'isDomain_cn',
       'isDomain_ca', 'isDomain_us', 'isDomain_ru', 'isDomain_uk',
       'isDomain_ch', 'isDomain_be', 'isDomain_de', 'isDomain_info',
       'isDomain_eu', 'isDomain_au', 'isDomain_tv', 'isDomain_es',
       'isDomain_gov', 'isDomain_it', 'domainCnt']
f_columns += ['n_sites', 'start_hour', 'is_h1', 'is_h2', 'is_h3', 'duration']

In [113]:
len(f_columns)

31

In [95]:
print(full_df.columns[20:])

Index(['time10', 'start_month', 'start_time', 'start_time_sin',
       'start_time_cos', 'domains', 'isDomain_com', 'isDomain_net',
       'isDomain_fr', 'isDomain_org', 'isDomain_edu', 'isDomain_cn',
       'isDomain_ca', 'isDomain_us', 'isDomain_ru', 'isDomain_uk',
       'isDomain_ch', 'isDomain_be', 'isDomain_de', 'isDomain_info',
       'isDomain_eu', 'isDomain_au', 'isDomain_tv', 'isDomain_es',
       'isDomain_gov', 'isDomain_it', 'domainCnt', 'finish_time', 'n_sites',
       'start_hour', 'is_h1', 'is_h2', 'is_h3', 'duration'],
      dtype='object')


In [135]:
# Ваш код здесь
full_sites_sparse_1 = hstack((full_sites_sparse, 
                              full_df[f_columns],
                              )).tocsr()

In [136]:
full_sites_sparse_1.shape

(336358, 48413)

In [137]:
X = full_sites_sparse_1[:idx_split,:]
ntr = int(X.shape[0] * 0.8)
print(full_sites_sparse_1.shape[0], ntr)
display(pd.Series(y_train[:ntr]).value_counts().reset_index())
display(pd.Series(y_train[ntr:]).value_counts().reset_index())

336358 202848


,index,0
0,0,200875
1,1,1973


,index,0
0,0,50389
1,1,324


In [149]:
#for ratio in [0.5, 0.6, 0.7, 0.8, 0.9]:
for ratio in [0.8]:
    print(ratio)
    #for C in [0.02, 0.03, 0.05, 0.1, 0.5, 1]:
    for C in [0.01, 0.05, 0.1, 0.2]:
        print(C, get_auc_lr_valid(full_df[:idx_split][f_columns].values, y_train, C=C, ratio=ratio, seed=17))
    print()

0.8
0.01 (0.93359720020218007, 0.82497073781678043)
0.05 (0.93584333637387551, 0.83493851248229445)
0.1 (0.93676003164515864, 0.83979804901957389)
0.2 (0.93642173764654191, 0.84221880136036875)



In [161]:
for ratio in [0.7, 0.8, 0.9]:
    print(ratio)
    #for C in [0.02, 0.03, 0.05, 0.1, 0.5, 1]:
    for C in [0.2, 0.5, 1.0]:
        print(C, get_auc_lr_valid(full_sites_sparse_1[:idx_split,:], y_train, C=C, ratio=ratio, seed=17))
    print()

0.7
0.2 (0.92748235344264585, 0.94854210744603817)
0.5 (0.93407313403252301, 0.97532650730159176)
1.0 (0.93740364917602204, 0.98466897818293853)

0.8
0.2 (0.97056499201643309, 0.94682391879672401)
0.5 (0.9736960031204146, 0.97463067654783253)
1.0 (0.97314504267906798, 0.9835435226837983)

0.9
0.2 (0.96680288719901364, 0.95039081436414652)
0.5 (0.96952458068107861, 0.97650123010778267)
1.0 (0.96872346990407809, 0.98466080808842971)



In [164]:
from sklearn.linear_model import SGDClassifier

In [168]:
for ratio in [0.8, 0.9]:
    print(ratio)
    #for C in [0.02, 0.03, 0.05, 0.1, 0.5, 1]:
    for C in [0.1, 0.5]:
        lr = SGDClassifier(loss='log',  max_iter=100, alpha=C * 1e-4)
        print(C, get_auc_lr_valid(full_sites_sparse_1[:idx_split,:], y_train, C=C, ratio=ratio, seed=17, lr=lr))
    print()

0.8
0.1 (0.97747279253824992, 0.97853428250895491)
0.5 (0.97713259973210886, 0.94942672185266508)

0.9
0.1 (0.9757022059755891, 0.97800464358372396)
0.5 (0.97389227226233566, 0.9496896931297556)



**Добавьте два новых признака: start_hour и morning.**

Признак `start_hour` – это час в который началась сессия (от 0 до 23), а бинарный признак `morning` равен 1, если сессия началась утром и 0, если сессия началась позже (будем считать, что утро это если `start_hour равен` 11 или меньше).

**Посчитйте ROC AUC на отложенной выборке для выборки с:**
- сайтами, `start_month` и `start_hour`
- сайтами, `start_month` и `morning`
- сайтами, `start_month`, `start_hour` и `morning`

In [187]:
# Ваш код здесь
#lr = LogisticRegression(C=0.5, random_state=17, penalty='l1', solver='liblinear', class_weight='balanced')
lr = LogisticRegression(C=0.5, random_state=17, penalty='l1', solver='liblinear')

lr = SGDClassifier(loss='log',  max_iter=100, alpha=1.0 * 1e-5, random_state=7, penalty='l1')
X = full_sites_sparse_1[:idx_split,:]
lr.fit(X, y_train)
pred_labels = lr.predict_proba(full_sites_sparse_1[idx_split:,:])[:, 1]

In [188]:
roc_auc_score(y_train, lr.predict_proba(full_sites_sparse_1[:idx_split,:])[:, 1])

0.97101674395726689

In [154]:
lr.intercept_

array([-0.90137795])

In [155]:
for i in np.argsort(np.abs(lr.coef_))[0][-30:]:
    print(i, lr.coef_[0, i])

17283 3.03074719616
18876 3.07039287181
397 3.08952141335
27298 3.1646255669
9783 3.2253223062
941 3.28834708232
270 3.3953493338
167 -3.41299298563
27307 3.42599830143
11241 3.51844469861
48384 -3.56760349203
1455 3.62385753801
7494 3.62492555551
77 3.6350538731
27269 3.65099235669
24553 3.68430903451
704 3.7131242086
4240 4.05498339399
265 4.10410660219
27189 4.63515193761
27184 4.83919338158
3000 4.99077728947
2078 4.99178794227
812 -4.99391852651
12619 5.15231651798
27174 5.88775869023
27344 6.10112102285
27190 6.10962196373
7832 6.36543578365
570 -6.49576795577


In [189]:
write_to_submission_file(pred_labels, test_df['session_id'], 'submissions/my_l1_1.0_sgd.txt')

In [119]:
len(pred_labels)

82797

### 4. Подбор коэффицициента регуляризации

Итак, мы ввели признаки, которые улучшают качество нашей модели по сравнению с первым бейслайном. Можем ли мы добиться большего значения метрики? После того, как мы сформировали обучающую и тестовую выборки, почти всегда имеет смысл подобрать оптимальные гиперпараметры -- характеристики модели, которые не изменяются во время обучения. Например, на 3 неделе вы проходили решающие деревья, глубина дерева это гиперпараметр, а признак, по которому происходит ветвление и его значение -- нет. В используемой нами логистической регрессии веса каждого признака изменяются и во время обучения находится их оптимальные значения, а коэффициент регуляризации остается постоянным. Это тот гиперпараметр, который мы сейчас будем оптимизировать.

Посчитайте качество на отложенной выборке с коэффициентом регуляризации, который по умолчанию `C=1`:

In [ ]:
# Ваш код здесь

Постараемся побить этот результат за счет оптимизации коэффициента регуляризации. Возьмем набор возможных значений C и для каждого из них посчитаем значение метрики на отложенной выборке.

Найдите `C` из `np.logspace(-3, 1, 10)`, при котором ROC AUC на отложенной выборке максимален. 

In [ ]:
# Ваш код здесь

Наконец, обучите модель с найденным оптимальным значением коэффициента регуляризации и с построенными признаками `start_hour`, `start_month` и `morning`. Если вы все сделали правильно и загрузите это решение, то повторите второй бенчмарк соревнования.

In [ ]:
# Ваш код здесь